# Face Detector (work in progress)
### Testing face detector using "face_angles.jpg"

In [6]:
# getting image path
# try with flowers.jpg as well (doesn't have any faces)
current = %pwd
from pathlib import Path
path = Path(current)
imagePath = str(path.parent.absolute())
image_name = "face_angles.jpg"
imagePath += '\\' + image_name
imagePath

'c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\face_angles.jpg'

In [7]:
cascPath = "haarcascade_frontalface_default.xml"
cascPath_m = "haarcascade_mouth.xml"
cascPath_e = "haarcascade_eye.xml"

In [8]:
import cv2

In [9]:
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)
mouthCascade = cv2.CascadeClassifier(cascPath_m)
eyeCascade = cv2.CascadeClassifier(cascPath_e)

# Read the image
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

mouths = mouthCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

eyes = eyeCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

In [10]:
print ("Found {0} faces!".format(len(faces)))

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

print ("Found {0} mouths!".format(len(mouths)))

# Draw a rectangle around the mouths
for (x, y, w, h) in mouths:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

print ("Found {0} eyes!".format(len(eyes)))

# Draw a rectangle around the eyes
for (x, y, w, h) in eyes:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)

Found 9 faces!
Found 22 mouths!
Found 13 eyes!


In [11]:
# input two rectangles
def temp_contains(R1, R2):

    # recalculating the sides from (x1, y1, width, height)
    # to (x1, y1, x2, y2)
    R1X2 = R1[0] + R1[2]
    R1Y2 = R1[1] + R1[3]
    R2X2 = R2[0] + R2[2]
    R2Y2 = R2[1] + R2[3]

    new_x1 = min(R1[0], R2[0])
    new_x2 = max(R1X2, R2X2) - new_x1 # width
    new_y1 = min(R1[1], R2[1])
    new_y2 = max(R1Y2, R2Y2) - new_y1 # height

    New_rect = [new_x1, new_y1, new_x2, new_y2]

    if (R1[0] >= R2X2) or (R1X2 <= R2[0]) or (R1Y2 <= R2[1]) or (R1[1] >= R2Y2):
        # not valid. not contained!
        return False, []
    else:
        # valid. contains
        return True, New_rect


In [12]:
for face in faces:
    for mouth in mouths:
        Valid, mesh_rect = temp_contains(face, mouth)
        if Valid:
            for eye in eyes:
                second_Valid, new_mesh_rect = temp_contains(mesh_rect, eye)
                if second_Valid:
                    x = new_mesh_rect[0]
                    y = new_mesh_rect[1]
                    w = new_mesh_rect[2]
                    h = new_mesh_rect[3]
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 2)

                


In [13]:
# runs reasonably well
# notice how it doesn't really work on faces that are turned 90 degrees.

cv2.imshow("Features found", image)
cv2.waitKey(0)


-1

# Face As A Percentage of the Video

In [59]:
# first, get the path of the video in question
# getting video path
current = %pwd
from pathlib import Path
path = Path(current)
filename = str(path.parent.absolute())
video_name = "American Grilled Cheese Vs. French Grilled Cheese.mp4"
# video_name = "Binging with Babish_ Patrick's Briefcase from SpongeBob SquarePants.mp4"
filename += '\\' + video_name
filename

'c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\American Grilled Cheese Vs. French Grilled Cheese.mp4'

In [60]:
import cv2
import numpy as np

In [61]:
# input two rectangles
def contains(R1, R2):

    # recalculating the sides from (x1, y1, width, height)
    # to (x1, y1, x2, y2)
    R1X2 = R1[0] + R1[2]
    R1Y2 = R1[1] + R1[3]
    R2X2 = R2[0] + R2[2]
    R2Y2 = R2[1] + R2[3]

    new_x1 = min(R1[0], R2[0])
    new_x2 = max(R1X2, R2X2) - new_x1 # width
    new_y1 = min(R1[1], R2[1])
    new_y2 = max(R1Y2, R2Y2) - new_y1 # height

    New_rect = [new_x1, new_y1, new_x2, new_y2]

    if (R1[0] >= R2X2) or (R1X2 <= R2[0]) or (R1Y2 <= R2[1]) or (R1[1] >= R2Y2):
        # not valid. not contained!
        return False, []
    else:
        # valid. contains
        return True, New_rect


In [62]:
def face_detector(image):

    max_pixels = 0
    return_me = False
    
    cascPath = "haarcascade_frontalface_default.xml"
    cascPath_m = "haarcascade_mouth.xml"
    cascPath_e = "haarcascade_eye.xml"

    faceCascade = cv2.CascadeClassifier(cascPath)
    mouthCascade = cv2.CascadeClassifier(cascPath_m)
    eyeCascade = cv2.CascadeClassifier(cascPath_e)

    # Read the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    mouths = mouthCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
    )

    eyes = eyeCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
    )

    # yuck. if you can make this faster, go ahead
    # makes sure the face, mouth, and eye rectangles contain each other.
    # This ensures a face
    for face in faces:
        for mouth in mouths:
            Valid, mesh_rect = contains(face, mouth)
            if Valid:
                for eye in eyes:
                    second_Valid, new_mesh_rect = contains(mesh_rect, eye)

                    if second_Valid:
                        return_me = True
                        width = new_mesh_rect[2]
                        height = new_mesh_rect[3]
                        if width * height > max_pixels:
                            max_pixels = width * height
                        # cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 2)
    
    return return_me, max_pixels * 100 / (image.shape[0] * image.shape[1])


In [63]:
# input: the video file
# output: length of video file in seconds

def find_video_length(video):

    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return frame_count / fps, fps


In [64]:
def face_percentage(filename):

    face_by_seconds = []
    face_size_of_screen = []

    # person_counter counts number of frames person was in shot
    person_counter = 0
    # no_person_counter counts number of frames where person is not in shot
    no_person_counter = 0

    my_video = cv2.VideoCapture(filename)
    if (my_video.isOpened() == False):
        print("Error opening video stream or file")

    _, video_fps = find_video_length(my_video)

    nth_frame = video_fps // 2
    frame_counter = 0

    while (my_video.isOpened()):
        
        ret, frame = my_video.read()
        if frame_counter % nth_frame == 0:

            # Capture frame-by-frame
            if ret == True:
                has_face, percentage = face_detector(frame)
                if has_face:
                    face_by_seconds.append(frame_counter / video_fps)
                    face_size_of_screen.append(percentage)
                    person_counter += 1
                else:
                    no_person_counter += 1
            else:
                break

        if ret == False:
            break

        frame_counter += 1
    
    percentage_of_face_in_video = (person_counter * 100) / (no_person_counter + person_counter)

    return percentage_of_face_in_video, face_by_seconds, face_size_of_screen


In [65]:
face_percentage, face_time, face_size = face_percentage(filename)

In [86]:
def cluster_by_seconds(face_time_arr, cluster_time):
    original_cluster_time = cluster_time
    count_me = False
    cluster_array = []
    counter = 0
    for time in face_time_arr:
        # print("cluster time is", cluster_time)
        # print("time is", time)
        print("counter is", counter)
        if time < cluster_time:
            counter += 1
        else:
            cluster_array.append(counter)
            counter = 0
            cluster_time += original_cluster_time
    return cluster_array

ten_sec_cluster = cluster_by_seconds(face_time, 10)
print(ten_sec_cluster)

counter is 0
counter is 1
counter is 2
counter is 3
counter is 4
counter is 5
counter is 6
counter is 7
counter is 8
counter is 9
counter is 10
counter is 0
counter is 1
counter is 2
counter is 0
counter is 1
counter is 2
counter is 3
counter is 4
counter is 5
counter is 6
counter is 7
counter is 8
counter is 0
counter is 1
counter is 2
counter is 3
counter is 4
counter is 5
counter is 6
counter is 7
counter is 8
counter is 9
counter is 10
counter is 11
counter is 0
counter is 1
counter is 2
counter is 3
counter is 4
counter is 5
counter is 6
counter is 7
counter is 8
counter is 9
counter is 10
counter is 11
counter is 0
counter is 1
counter is 2
counter is 3
counter is 4
counter is 5
counter is 6
counter is 7
counter is 8
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter is 0
counter

In [87]:
face_time

[0.46713333333333334,
 0.9342666666666667,
 1.4014,
 1.8685333333333334,
 2.8028,
 3.2699333333333334,
 3.7370666666666668,
 4.2042,
 4.671333333333333,
 5.138466666666667,
 18.685333333333332,
 19.152466666666665,
 19.619600000000002,
 23.823800000000002,
 24.290933333333335,
 24.758066666666668,
 25.2252,
 25.692333333333334,
 28.495133333333335,
 28.962266666666668,
 29.4294,
 29.896533333333334,
 30.8308,
 31.297933333333333,
 31.765066666666666,
 35.035000000000004,
 35.50213333333333,
 35.96926666666667,
 36.4364,
 36.903533333333336,
 37.370666666666665,
 37.8378,
 39.239200000000004,
 39.70633333333333,
 40.17346666666667,
 41.107733333333336,
 41.574866666666665,
 43.443400000000004,
 43.91053333333333,
 44.37766666666667,
 44.8448,
 45.311933333333336,
 47.18046666666667,
 47.647600000000004,
 48.114733333333334,
 49.983266666666665,
 51.851800000000004,
 52.318933333333334,
 53.720333333333336,
 54.187466666666666,
 56.056000000000004,
 56.523133333333334,
 56.99026666666667